In [1]:
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
from keras.models import model_from_json
from keras.datasets import cifar10
from PIL import Image
from urllib.request import urlopen

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

categories = ['airplane', 'cars', 'bird', 'cats', 'deer', 'dog', 'frog', 'horse', 'ships', 'trucks']

def loadModel(json_desc, weights):
    # load json and create model
    json_file = open(json_desc, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(weights)
    return loaded_model
    
model = loadModel('models/allcnn_90_model.json', 'models/allcnn_90_weights.hdf5')

def transformImage(image):
    imgarr = np.asarray(image)
    imgarr = np.reshape(imgarr, (1, 32, 32, 3))
    imgarr = imgarr.astype('float32')
    imgarr /= 255
    return imgarr
    
def predictImage(image):    
    array = transformImage(image)
    result = model.predict(array)
    result = result[0].tolist()
    best_index=result.index(max(result))
    best_prob = result[best_index]
    
    result[best_index] = 0    
    sec_best_index=result.index(max(result))
    sec_best_prob = result[sec_best_index]

    result[sec_best_index] = 0    
    thd_best_index=result.index(max(result))
    thd_best_prob = result[thd_best_index]
    
    return best_prob, best_index, sec_best_prob, sec_best_index, thd_best_prob, thd_best_index

def processImage(url: str, focus :int):
    img = Image.open(urlopen(url))
    width, height = img.size
    if ( ((width/height) > 1.3) | ((width/height) > 0.7)):
        #img = cropImage(img, focus)
        img = squashImage(img)
    else:
        img = squashImage(img)
    return img

def squashImage(img: Image):
    print ("squash")
    imgsquash = img.resize((32, 32))     
    return imgsquash;

def cropImage(img: Image, focus :int):    
    left = 0
    top = 0
    right = 0
    bottom = 0
    imagetype = ""
    
    width, height = img.size
    maxdim = max(width, height)
    mindim = min(width, height)
    size = round(mindim - (mindim/(100/focus)))

    if (width > height) :
        imagetype = "landscape"
    else:
        imagetype = "portrait"
    
    left = round((width - size)/2) - 1
    top = round((height - size)/2) - 1
    right = left + size
    bottom = top + size
    print ("crop : "+imagetype+"("+str(width)+"x"+str(height)+") size : "+str(size)+"crop : ("+str(left)+","+str(top)+","+str(right)+","+str(bottom)+")")
    imgcrop = img.crop((left, top, right, bottom))
    imgcrop = imgcrop.resize((32, 32))
    return imgcrop

Using TensorFlow backend.


In [4]:
categories = ['airplane', 'cars', 'bird', 'cats', 'deer', 'dogs', 'frog', 'horse', 'ships', 'trucks']
urlbase = "https://jndfiles-pub.s3-eu-west-1.amazonaws.com/images"
category = "airplane"
correct = 0
count = 0
correctImages = []  
for i in range(1, 50):
    count = count + 1
    url = urlbase+"/"+category+"/"+category+"-"+str(i)+".jpg"
    #print (url)
    img = processImage(url, 1)
    best_prob, best_index, sec_best_prob, sec_best_index, thd_best_prob, thd_best_index = predictImage(img)
    if ( (str(categories[best_index]) == category) ):
        img = Image.open(urlopen(url))
        correct = correct + 1
        print(category+"-"+str(i)+".jpg : category : "+category+" prediction : "+str(categories[best_index]+" CORRECT"))
        correctImages.append(category+"-"+str(i)+".jpg")
    else:    
        print(category+"-"+str(i)+".jpg : category : "+category+" prediction : "+str(categories[best_index]))
    
#plt.show()    
print ("Count : "+str(count)+" correct :"+str(correct))  
print ("Correct images : "+str(correctImages))

squash
airplane-1.jpg : category : airplane prediction : airplane CORRECT
squash
airplane-2.jpg : category : airplane prediction : airplane CORRECT
squash
airplane-3.jpg : category : airplane prediction : airplane CORRECT
squash
airplane-4.jpg : category : airplane prediction : airplane CORRECT
squash
airplane-5.jpg : category : airplane prediction : airplane CORRECT
squash
airplane-6.jpg : category : airplane prediction : airplane CORRECT
squash
airplane-7.jpg : category : airplane prediction : airplane CORRECT
squash
airplane-8.jpg : category : airplane prediction : trucks
squash
airplane-9.jpg : category : airplane prediction : airplane CORRECT
squash
airplane-10.jpg : category : airplane prediction : airplane CORRECT
squash
airplane-11.jpg : category : airplane prediction : airplane CORRECT
squash
airplane-12.jpg : category : airplane prediction : airplane CORRECT
squash
airplane-13.jpg : category : airplane prediction : airplane CORRECT
squash
airplane-14.jpg : category : airplane

In [5]:
# fig,axes = plt.subplots(nrows = 3, ncols = 1, figsize=(30,30))

# for ax in axes.flatten():
#     ax.axis('off')
    
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print('X_test shape:', X_test.shape)
print('X_test[0] shape:', X_test[1].shape)
print(X_test.shape[0], 'test samples')
correct = 0
for i in range(0, 1000):
    data = X_test[i]
    label = y_test[i]
    data = np.reshape(data, (1, 32, 32, 3))
    data = data.astype('float32')
    data /= 255
    #print('data[0] shape:', data.shape)

    result = model.predict(data)
    result = result[0].tolist()
    best_index=result.index(max(result))
    best_prob = result[best_index]
    
    print ("prediction : "+str(categories[best_index])+" actual : "+str(categories[int(label)]))
    
    if (label == best_index):
        correct = correct + 1

print ("Count : "+str(1000)+" correct :"+str(correct))         

X_test shape: (10000, 32, 32, 3)
X_test[0] shape: (32, 32, 3)
10000 test samples
prediction : cats actual : cats
prediction : ships actual : ships
prediction : ships actual : ships
prediction : airplane actual : airplane
prediction : frog actual : frog
prediction : frog actual : frog
prediction : cars actual : cars
prediction : frog actual : frog
prediction : cats actual : cats
prediction : cars actual : cars
prediction : airplane actual : airplane
prediction : trucks actual : trucks
prediction : dogs actual : dogs
prediction : horse actual : horse
prediction : trucks actual : trucks
prediction : ships actual : ships
prediction : dogs actual : dogs
prediction : horse actual : horse
prediction : ships actual : ships
prediction : frog actual : frog
prediction : horse actual : horse
prediction : airplane actual : airplane
prediction : deer actual : deer
prediction : trucks actual : trucks
prediction : dogs actual : dogs
prediction : dogs actual : bird
prediction : deer actual : deer
predi